In [ ]:
%cd /content
!rm -rf GFPGAN
!git clone https://github.com/TencentARC/GFPGAN.git
%cd GFPGAN

# Set up the environment
# Install basicsr - https://github.com/xinntao/BasicSR
# We use BasicSR for both training and inference
!pip install basicsr
# Install facexlib - https://github.com/xinntao/facexlib
# We use face detection and face restoration helper in the facexlib package
!pip install facexlib
# Install other depencencies
!pip install -r requirements.txt
!python setup.py develop
!pip install realesrgan  # used for enhancing the background (non-face) regions
# Download the pre-trained model
!wget https://github.com/TencentARC/GFPGAN/releases/download/v0.2.0/GFPGANCleanv1-NoCE-C2.pth -P experiments/pretrained_models
# Now we use the V1.3 model for the demo
#!wget https://github.com/TencentARC/GFPGAN/releases/download/v1.3.0/GFPGANv1.3.pth -P experiments/pretrained_models

/content
Cloning into 'GFPGAN'...
remote: Enumerating objects: 523, done.
remote: Counting objects: 100% (210/210), done.
remote: Compressing objects: 100% (57/57), done.
remote: Total 523 (delta 168), reused 153 (delta 153), pack-reused 313
Receiving objects: 100% (523/523), 5.37 MiB | 20.01 MiB/s, done.
Resolving deltas: 100% (279/279), done.
/content/GFPGAN
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 172.5/172.5 kB 1.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 299.2/299.2 kB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 17.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 254.7/254.7 kB 20.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 25.9 MB/s eta 0:00:00
  Created wheel for basicsr: filename=basicsr-1.4.2-py3-none-any.whl size=214818 sha256=7907214f9f5ae85c2065ef1d6cea2bed6331c5aaae3c9c94883fe90e531506ad
  Stored in directory: /root/.cache/pi

# 2. This section is for creating some important folders



*   First Run this cell

*   Once you've run the cell , you can see that a folder named "videos" will be created inside the GFPGAN folder


*   Just drag and drop any number of videos from your computer and place them inside the "videos" folder. (videos you want to upscale)







In [ ]:
import os
from google.colab import files
import shutil

upload_folder = 'upload'
result_folder = 'results'
video_folder = 'videos'
video_result_folder = 'results_videos'
video_mp4_result_folder = 'results_mp4_videos'

if os.path.isdir(upload_folder):
  print(upload_folder+" exists")
else :
  os.mkdir(upload_folder)

if os.path.isdir(video_folder):
  print(video_folder+" exists")
else :
  os.mkdir(video_folder)

if os.path.isdir(video_result_folder):
  print(video_result_folder+" exists")
else :
  os.mkdir(video_result_folder)

if os.path.isdir(video_mp4_result_folder):
  print(video_mp4_result_folder+" exists")
else :
  os.mkdir(video_mp4_result_folder)

if os.path.isdir(result_folder):
  print(result_folder+" exists")

else :
  os.mkdir(result_folder)




# 3. Inference



*   Then run this cell , it will take a good amount of time depending on the length of your video and how many videos you want to upscale

*   Once the upscaling process is complete , you can find your upscaled video results , under the "results_mp4_videos" folder.









In [ ]:
import cv2
import numpy as np
import glob
from os.path import isfile, join
import subprocess
from IPython.display import clear_output

# assign directory
directory = '/content/GFPGAN/videos' #PATH_WITH_INPUT_VIDEOS
zee = 0

#deletes frames from previous video
for f in os.listdir(upload_folder):
    os.remove(os.path.join(upload_folder, f))

#deletes upscaled frames from previous video
#for f in os.listdir('results/restored_imgs'):
#    os.remove(os.path.join('results/restored_imgs', f))

#clearing previous .avi files
for f in os.listdir(video_result_folder):
    os.remove(os.path.join(video_result_folder, f))

#clearing .mp4 result files
for f in os.listdir(video_mp4_result_folder):
    os.remove(os.path.join(video_mp4_result_folder, f))


def convert_frames_to_video(pathIn,pathOut,fps):
    frame_array = []
    files = [f for f in os.listdir(pathIn) if isfile(join(pathIn, f))]
    #for sorting the file names properly
    files.sort(key = lambda x: int(x[5:-4]))
    size2 = (0,0)

    for i in range(len(files)):
        filename=pathIn + files[i]
        #reading each files
        img = cv2.imread(filename)
        height, width, layers = img.shape
        size = (width,height)
        size2 = size
        print(filename)
        #inserting the frames into an image array
        frame_array.append(img)
    out = cv2.VideoWriter(pathOut,cv2.VideoWriter_fourcc(*'DIVX'), fps, size2)
    for i in range(len(frame_array)):
        # writing to a image array
        out.write(frame_array[i])
    out.release()


for filename in os.listdir(directory):

    f = os.path.join(directory, filename)
    # checking if it is a file
    if os.path.isfile(f):


      print("PROCESSING :"+str(f)+"\n")
      # Read the video from specified path

      #video to frames
      cam = cv2.VideoCapture(str(f))

      try:

          # PATH TO STORE VIDEO FRAMES
          if not os.path.exists('/content/GFPGAN/upload'):
              os.makedirs('/content/GFPGAN/upload')

      # if not created then raise error
      except OSError:
          print ('Error: Creating directory of data')

      # frame
      currentframe = 0

      #clear all folders


      #deletes upscaled frames from previous video
      #for f in os.listdir(result_folder):
      #  os.remove(os.path.join(result_folder, f))




      while(True):

          # reading from frame
          ret,frame = cam.read()

          if ret:
              # if video is still left continue creating images
              name = '/content/GFPGAN/upload/frame' + str(currentframe) + '.jpg'

              # writing the extracted images
              cv2.imwrite(name, frame)


                # increasing counter so that it will
                # show how many frames are created
              currentframe += 1
              print(currentframe)
          else:
              #deletes all the videos you uploaded for upscaling
              #for f in os.listdir(video_folder):
              #  os.remove(os.path.join(video_folder, f))

              break

        # Release all space and windows once done
      cam.release()
      cv2.destroyAllWindows()

      #apply super-resolution on all frames of a video
      #scale factor is by 3.5x

      #in the line below '2' stands for upscaling by factor of 2
      !python inference_gfpgan.py -i upload -o results -v 1.2 -s 2 --bg_upsampler realesrgan

      #after upscaling just delete the source frames
      for f in os.listdir(upload_folder):
          os.remove(os.path.join(upload_folder, f))

      '''
      #rename all frames in "results" to remove the 'out' substring from the processing results
      paths = (os.path.join(root, filename)
              for root, _, filenames in os.walk('/content/GFPGAN/results')
              for filename in filenames)
      for path in paths:
          newname = path.replace('_out', '')
          if newname != path:
              os.rename(path, newname)
      '''

      #convert super res frames to .avi
      pathIn = '/content/GFPGAN/results/restored_imgs/'

      zee = zee+1
      fName = "video"+str(zee)
      filenameVid = f"{fName}.avi"

      pathOut = "/content/GFPGAN/results_videos/"+filenameVid

      fps = 25.0 #change this to FPS of your source video

      convert_frames_to_video(pathIn, pathOut, fps)

      #after processing frames converted to .avi video , delete upscaled frames from previous video
      for f in os.listdir('results/restored_imgs'):
          os.remove(os.path.join('results/restored_imgs', f))

      #convert .avi to .mp4
      src = '/content/GFPGAN/results_videos/'
      dst = '/content/GFPGAN/results_mp4_videos/'

      for root, dirs, filenames in os.walk(src, topdown=False):
          #print(filenames)
          for filename in filenames:
              print('[INFO] 1',filename)
              try:
                  _format = ''
                  if ".flv" in filename.lower():
                      _format=".flv"
                  if ".mp4" in filename.lower():
                      _format=".mp4"
                  if ".avi" in filename.lower():
                      _format=".avi"
                  if ".mov" in filename.lower():
                      _format=".mov"

                  inputfile = os.path.join(root, filename)
                  print('[INFO] 1',inputfile)
                  outputfile = os.path.join(dst, filename.lower().replace(_format, ".mp4"))
                  subprocess.call(['ffmpeg', '-i', inputfile, outputfile])
              except:
                  print("An exception occurred")

      clear_output(wait=True)

      #clearing previous .avi files
      for f in os.listdir(video_result_folder):
          os.remove(os.path.join(video_result_folder, f))

      #deletes frames from previous video
      #for f in os.listdir(upload_folder):
      #  os.remove(os.path.join(upload_folder, f))



      # if it is out of memory, try to use the `--tile` option
# We upsample the image with the scale factor X3.5

# Arguments
# -n, --model_name: Model names
# -i, --input: input folder or image
# --outscale: Output scale, can be arbitrary scale factore.

PROCESSING :/content/GFPGAN/videos/test.mp4

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
/usr/local/lib/python3.10/dist-packages/torchvision/transforms/functional_tensor.py:5: UserWarning: The torchvision.transforms.functional_tensor module is deprecated in 0.15 and will be **removed in 0.17**. Please don't rely on it. You probably just need to use APIs in torchvision.transforms.functional or in torchvision.transforms.v2.functional.
  warnings.warn(
Downloading: "https://

# 4. Download Results



*   All your upscaled .mp4 files will be inside the results_mp4_videos folder (inside GFPGAN folder)


*   You can right click on the needed file and download it from there.






# 5. Run this cell after the batch of videos have been upscaled
## ( Deletes processed data)






In [ ]:


#deletes frames from previous video
for f in os.listdir(upload_folder):
    os.remove(os.path.join(upload_folder, f))

#deletes upscaled frames from previous video
for f in os.listdir('results/restored_imgs'):
    os.remove(os.path.join('results/restored_imgs', f))

#deletes all the videos you uploaded for upscaling
for f in os.listdir(video_folder):
    os.remove(os.path.join(video_folder, f))

#clearing previous .avi files
for f in os.listdir(video_result_folder):
    os.remove(os.path.join(video_result_folder, f))

#clearing .mp4 result files
for f in os.listdir(video_mp4_result_folder):
    os.remove(os.path.join(video_mp4_result_folder, f))